In [ ]:
from flask import Flask
import numpy as np
import cv2
import tensorflow as tf;
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.3.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
#from google.colab import files

#content_img = files.upload()

In [ ]:
#from google.colab import files
#styleImage = files.upload()

In [ ]:
def get_image(imgpath):
    w=512
    h=512
    loaded_img = tf.keras.preprocessing.image.load_img(imgpath,target_size=(w,h))
   
    image = tf.keras.applications.vgg19.preprocess_input(tf.keras.preprocessing.image.img_to_array(loaded_img))
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = np.expand_dims(image , axis = 0)
    return image
def imgshow(image):
    image[:,:,0]  *=64/255.0
    image[:,:,1] *=128/255.0
    image[:,:,2] *=192/255.0
    image = image[:,:,::-1]
    image = np.clip(image,0,255).astype('uint8')
    img=np.squeeze(image)
    plt.figure(figsize = (50,10))
    plt.imshow(img, interpolation='nearest')
def imshowImage(image):
  image[:,:,0]  *=64/255.0
  image[:,:,1] *=128/255.0
  image[:,:,2] *=192/255.0
  image = image[:,:,::-1]
  image = np.clip(image,0,255).astype('uint8')
  img=np.squeeze(image)
  return img
#styleImage = get_image("/content/style.jpg")
#contentImage = get_image("/content/content.jpg")
#imgshow(styleImage)


In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

# Layer Extraction From VGG Model 

In [ ]:
baseModel = VGG19(weights='imagenet',include_top=False,pooling='Max');
baseModel.trainable = False;
contentImageLayers = ['block5_conv2']
styleImageLayers = ["block1_conv1","block2_conv1","block3_conv1","block4_conv1","block5_conv1"]
baseModel.get_layer(contentImageLayers[0])
print(baseModel.input)

Tensor("input_2:0", shape=(None, None, None, 3), dtype=float32)


#Model Prep

In [ ]:
contentModel = Model(inputs = baseModel.input,outputs = baseModel.get_layer(contentImageLayers[0]).output) 

In [ ]:
styleModel = [Model(inputs = baseModel.input , outputs = baseModel.get_layer(Layer).output) for Layer in styleImageLayers]

# Content Loss

In [ ]:
def customLoss(model,inputImage,outputImage):
  val = tf.reduce_mean((model(inputImage) - model(outputImage))**2)
  return val;

# GRAM Matrix ( For Style Loss)

In [ ]:
def gramMatrix(inputImage):
  channels=int(inputImage.shape[-1])
  a=tf.reshape(inputImage,[-1,channels])
  n=tf.shape(a)[0]
  gram=tf.matmul(a,a,transpose_a=True)
  return(gram/tf.cast(n,tf.float32))

# Style Loss

In [ ]:
def styleLoss(model,inputImage,generated):
  error = 0
  for styleModels in model:
    styleVal = styleModels(inputImage);
    generatedVal = styleModels(generated)
    val = tf.reduce_mean((gramMatrix(styleVal) - gramMatrix(generatedVal))**2);
    error = error + val*1/len(model);
  return error;

# Training the Initial Image with Content and Style Images

In [ ]:
def runFunction(contentImage,styleImage):
  ## Calculate loss for content image
  ## Calculate loss for style image
  ## update the initial image by the error from the losses calculated
  ## repeat the process
  contentGenerated = tf.Variable(contentImage,dtype = tf.float32);
  optimizer = tf.keras.optimizers.Adam(learning_rate = 10)
  maxImage = contentGenerated;
  maxCost = 1e12

  ## Weighted Loss calcualtion
  for i in range(0,500):
    print('Iteration : ' + str(i))
    with tf.GradientTape() as tape:
      contentLossVal = customLoss(contentModel,contentImage,contentGenerated)
      styleLossVal = styleLoss(styleModel , styleImage , contentGenerated)
      cost = contentLossVal * 40 + styleLossVal * 1000;
      gradient = tape.gradient(cost , contentGenerated);
      optimizer.apply_gradients([(gradient,contentGenerated)])
      print(cost)
      if(cost < maxCost):
        maxCost = cost;
        maxImage = contentGenerated.numpy();
  return maxImage,contentGenerated.numpy();

In [ ]:
#img,imgVal = runFunction(contentImage,styleImage)

In [ ]:
## Original Image

#imgshow(contentImage)

# Result Image

In [ ]:
#imgshow(imgVal)

In [ ]:
!pip install flask-ngrok

In [ ]:
## FLASK
##  Hosted on ngrok :: http://f211c07033ea.ngrok.io
import io
from PIL import Image
import os
from flask import request, Response
from flask import render_template
from flask import send_file
from flask_ngrok import run_with_ngrok

## The HTML Pages are stored in tempelate folder 
folder = "/content/templates";

## Direct folder where uploaded image will be saved
app = Flask(__name__)
app.config['folder'] = folder
run_with_ngrok(app)  ## Start ngork

@app.route('/',methods = ['POST' , 'GET'])   ## First Page

## Sending Image to the server
def neuralArt():
  return "<H1> Welcome to Neural Art Tranfer</H1>"


## Getting image from the client
@app.route('/imageInput' , methods = ['POST' ,'GET'])  ## Second Page
def getImage():
  if(request.method == 'POST'):   ## Check type of request  
                                                                ##POST
  ##  Style Image
    file = request.files['styleImage']
    if file:
      filename = file.filename
      file.save(os.path.join(app.config['folder'], filename))


## Content Image
    file = request.files['contentImage']
    if file:
      filename = file.filename
      file.save(os.path.join(app.config['folder'], filename))

## Getting the images from local folders(Remember to run above cells so the get_image func are available)
    contentImage = get_image("/content/templates/content.jpg")
    styleImage = get_image("/content/templates/style.jpg")


    _ , resImage = runFunction(contentImage,styleImage)

## We need to show a waiting picture till the processing is done


##  styleTranferred Image is displayed

    content_type = 'image/jpeg'
    file_object = io.BytesIO()
    img = imshowImage(resImage);
    img = Image.fromarray(img.astype('uint8'))
    img.save(file_object, 'jpeg')
    file_object.seek(0)
    return send_file(file_object , mimetype = 'image/jpeg')


  else:                                                     ## GET 
    return render_template('imageUpload.html')

if __name__ == '__main__':
  app.run()
  

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d922a1b70620.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Oct/2020 10:45:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 10:45:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Oct/2020 10:45:10] "GET /imageInput HTTP/1.1" 200 -


Iteration : 0
tf.Tensor(20957123000000.0, shape=(), dtype=float32)
Iteration : 1
tf.Tensor(10869095000000.0, shape=(), dtype=float32)
Iteration : 2
tf.Tensor(8850703000000.0, shape=(), dtype=float32)
Iteration : 3
tf.Tensor(7183140000000.0, shape=(), dtype=float32)
Iteration : 4
tf.Tensor(6062551000000.0, shape=(), dtype=float32)
Iteration : 5
tf.Tensor(5042253000000.0, shape=(), dtype=float32)
Iteration : 6
tf.Tensor(4281331000000.0, shape=(), dtype=float32)
Iteration : 7
tf.Tensor(3798736000000.0, shape=(), dtype=float32)
Iteration : 8
tf.Tensor(3375576600000.0, shape=(), dtype=float32)
Iteration : 9
tf.Tensor(3030325100000.0, shape=(), dtype=float32)
Iteration : 10
tf.Tensor(2734189400000.0, shape=(), dtype=float32)
Iteration : 11
tf.Tensor(2475652500000.0, shape=(), dtype=float32)
Iteration : 12
tf.Tensor(2249810500000.0, shape=(), dtype=float32)
Iteration : 13
tf.Tensor(2057380700000.0, shape=(), dtype=float32)
Iteration : 14
tf.Tensor(1882758600000.0, shape=(), dtype=float32)
Ite

127.0.0.1 - - [20/Oct/2020 10:49:43] "POST /imageInput HTTP/1.1" 200 -


tf.Tensor(586719800000.0, shape=(), dtype=float32)


127.0.0.1 - - [20/Oct/2020 10:49:45] "GET /favicon.ico HTTP/1.1" 404 -
